In [1]:
import sys
if '..' not in sys.path:
    sys.path.append('..')
import pandas as pd
from sklearn.metrics import fbeta_score
import numpy as np

import dataset

In [10]:
def f2_score(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2, average='samples')

In [67]:
pred_df = pd.read_csv('../runs/resnet50-aug/valid.csv', index_col=0)
pred_df = pred_df.groupby(level=0).mean()
pred_data = pred_df.as_matrix()

train_df = pd.read_csv('../data/train_v2.csv', index_col=0)
train_data = np.zeros((len(pred_df), dataset.N_CLASSES), dtype=np.uint8)
for i, tags in enumerate(train_df.loc[pred_df.index]['tags']):
    for tag in tags.split():
        train_data[i, dataset.CLASSES.index(tag)] = 1
pred_df[:5]

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
train_1,0.832206,0.000445,0.004546,0.000257,0.000190,0.991775,3.623631e-07,0.000045,0.165169,0.086677,0.000172,0.006395,0.999855,0.492553,0.001076,0.002657,0.819770
train_1000,0.028228,0.000038,0.000195,0.000218,0.000254,0.000044,3.386253e-05,0.000014,0.007382,0.001181,0.000011,0.999485,0.999967,0.014324,0.000314,0.000432,0.039293
train_10002,0.993809,0.000067,0.012697,0.000022,0.000020,0.990278,1.564288e-05,0.000094,0.111505,0.114115,0.002636,0.007507,0.957796,0.965091,0.000042,0.001357,0.097161
train_10008,0.011744,0.996610,0.131748,0.000294,0.000009,0.982018,1.218989e-04,0.027884,0.015543,0.040410,0.005883,0.007961,0.965080,0.283321,0.000547,0.000251,0.972098
train_10017,0.986268,0.000118,0.020832,0.000059,0.000015,0.954083,4.547439e-06,0.001013,0.094858,0.485867,0.015214,0.025627,0.968892,0.994855,0.001385,0.001317,0.223070


In [69]:
f2_score(train_data, pred_data > 0.2)

0.9226078512627045

In [70]:
def optimise_f2_thresholds(y, p, verbose=True, resolution=100):
    def mf(x):
        p2 = np.zeros_like(p)
        for i in range(17):
            p2[:, i] = (p[:, i] > x[i]).astype(np.int)
        score = fbeta_score(y, p2, beta=2, average='samples')
        return score

    x = [0.2] * y.shape[1]
    for i in range(y.shape[1]):
        best_i2 = 0
        best_score = 0
        for i2 in range(resolution):
            i2 /= resolution
            x[i] = i2
            score = mf(x)
            if score > best_score:
                best_i2 = i2
                best_score = score
        x[i] = best_i2
        if verbose:
            print(i, best_i2, best_score)

    return x

optimise_f2_thresholds(train_data, pred_data)

0 0.17 0.922742443144
1 0.25 0.922780120689
2 0.22 0.92280361198
3 0.23 0.922865437711
4 0.33 0.922888795242
5 0.19 0.922948694288


/home/kostia/ml/kaggle-planet-2017/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


6 0.09 0.923252513721
7 0.18 0.923276575628
8 0.21 0.923403029709
9 0.18 0.923404160587
10 0.23 0.92344969275
11 0.2 0.92344969275
12 0.25 0.923513481427
13 0.23 0.923708107382
14 0.1 0.923795720452
15 0.25 0.923808413894
16 0.17 0.924069806579


[0.17,
 0.25,
 0.22,
 0.23,
 0.33,
 0.19,
 0.09,
 0.18,
 0.21,
 0.18,
 0.23,
 0.2,
 0.25,
 0.23,
 0.1,
 0.25,
 0.17]

In [71]:
optimise_f2_thresholds(train_data, pred_data, resolution=500)

0 0.174 0.922769841761
1 0.254 0.922816316869
2 0.224 0.922869593825
3 0.218 0.922936766647
4 0.326 0.922960124178
5 0.19 0.923020023223


/home/kostia/ml/kaggle-planet-2017/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


6 0.128 0.923361480165
7 0.176 0.923385542071
8 0.216 0.923569960113
9 0.188 0.923596840679
10 0.234 0.923664888205
11 0.202 0.923676117095
12 0.25 0.923739905772
13 0.246 0.923940528208
14 0.104 0.924077437002
15 0.134 0.92410002247
16 0.172 0.924399369658


[0.174,
 0.254,
 0.224,
 0.218,
 0.326,
 0.19,
 0.128,
 0.176,
 0.216,
 0.188,
 0.234,
 0.202,
 0.25,
 0.246,
 0.104,
 0.134,
 0.172]

In [74]:
pred2_df = pd.read_csv('../runs/resnet50-aug-fold2/valid.csv', index_col=0)
pred2_df = pred2_df.groupby(level=0).mean()
pred2_data = pred2_df.as_matrix()
assert pred2_df.index.equals(pred_df.index)

In [76]:
f2_score(train_data, pred2_data > 0.2)

0.92745234222823647

In [78]:
np.mean([f2_score(train_data, pred_data > 0.2), f2_score(train_data, pred2_data > 0.2)])

0.92503009674547054

In [75]:
optimise_f2_thresholds(train_data, pred2_data)

0 0.2 0.927452342228
1 0.11 0.927572136469
2 0.15 0.927585958201
3 0.19 0.927652997346
4 0.29 0.927673410649
5 0.29 0.92782789578


/home/kostia/ml/kaggle-planet-2017/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


6 0.15 0.92825963928
7 0.08 0.928301335224
8 0.19 0.928383085237
9 0.19 0.928400222428
10 0.2 0.928400222428
11 0.22 0.928458333027
12 0.35 0.928597845902
13 0.26 0.928890983868
14 0.19 0.928934575222
15 0.12 0.928967602457
16 0.19 0.929030713267


[0.2,
 0.11,
 0.15,
 0.19,
 0.29,
 0.29,
 0.15,
 0.08,
 0.19,
 0.19,
 0.2,
 0.22,
 0.35,
 0.26,
 0.19,
 0.12,
 0.19]